In [1]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords

In [2]:
train_data=pd.read_csv('./f2c2f440-8-dataset_he/train.csv')
test_data = pd.read_csv('./f2c2f440-8-dataset_he/test.csv')

In [3]:
def changeToBinary(x):
    if x == 'not happy':
        return 0
    else :
        return 1

In [4]:
train_data.Is_Response = train_data.Is_Response.apply(changeToBinary)

In [5]:
train_data.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,0
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,0
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,0
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,1
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,0


In [6]:
text_reviews , y_train = train_data.Description , train_data.Is_Response

In [24]:
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
stop_words = stopwords.words('english') + list(punctuation)
def tokenize(text):
    words = word_tokenize(text)
    words = [w.lower() for w in words]
    return [w for w in words if w not in stop_words and not w.isdigit()]

In [7]:
#Count Vectorizer ( Bag of Words)
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidf = TfidfVectorizer(min_df=5, norm=None , ngram_range= (1,3))
tfidf.fit((train_data['Description']))
train_tf= tfidf.transform(train_data['Description'])
test_tf = tfidf.transform(test_data['Description'])
#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [26]:
X_train = train_tf.toarray()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import GaussianNB

clf = LogisticRegression()
scores = cross_val_score(clf , train_tf.toarray(), y_train , cv=5)
#clf.fit(X_train_counts , y_train)

In [15]:
np.mean(scores)

0.81891482493105572

In [ ]:
#Logistic  - 83.86%
# MultinomialNB - 85.83

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
pipe = make_pipeline(TfidfVectorizer(), LogisticRegression())
param_grid = {'logisticregression__C': [ 0.01],
              "tfidfvectorizer__ngram_range": [(1, 3)]}
grid = GridSearchCV(pipe, param_grid , cv=5)
grid.fit(text_reviews, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smoo...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidfvectorizer__ngram_range': [(1, 3)], 'logisticregression__C': [0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [8]:
grid.best_score_

0.68121339771909994

In [42]:
grid.best_params_

{'linearsvc__C': 0.0001, 'tfidfvectorizer__ngram_range': (1, 3)}

In [68]:
test_reviews = test_data.Description
len(test_reviews)

29404

In [76]:
#test_tfid_counts = count_vect.transform(test_reviews)
#test_tfid_vectors = tfidf_transformer.transform(test_tfid_counts)

X_test_counts = count_vect.transform(test_reviews)

In [77]:
X_test_counts

<29404x128611 sparse matrix of type '<type 'numpy.float64'>'
	with 6039800 stored elements in Compressed Sparse Row format>

In [81]:
test_response = clf.predict(X_test_counts)

In [79]:
#test_response = grid.predict(test_reviews)

In [82]:
test_data['Is_Response'] = test_response

In [83]:
def changeToString(x):
    if x == 0:
        return 'not_happy'
    else :
        return 'happy'

In [84]:
test_data.Is_Response = test_data.Is_Response.apply(changeToString)

In [85]:
test_data[['User_ID' ,'Is_Response']].to_csv('./IsHappy.csv')

In [20]:
test_data.to_csv('./Test.csv')

In [33]:
teeting = pd.read_csv('./Test.csv')


In [37]:
(teeting[teeting.Is_Response != teeting.Is_Response_2]).to_csv('Unmatched.csv')